# Working with Environment Variables

- Environment variables are dynamic, named values provided by the operating system to running processes, enabling configuration of behavior without code modifications.
- They allow applications to adapt across development, staging, and production environments by externalizing configuration data such as API keys, file paths, and feature flags.
- Python’s `os` module offers simple interfaces to access and manage these variables, promoting separation of code and configuration.

In [23]:
pip list

Package                   Version
------------------------- -----------
anyio                     4.12.0
argon2-cffi               25.1.0
argon2-cffi-bindings      25.1.0
arrow                     1.4.0
asttokens                 3.0.1
async-lru                 2.0.5
attrs                     25.4.0
babel                     2.17.0
beautifulsoup4            4.14.3
bleach                    6.3.0
certifi                   2025.11.12
cffi                      2.0.0
charset-normalizer        3.4.4
comm                      0.2.3
debugpy                   1.8.18
decorator                 5.2.1
defusedxml                0.7.1
executing                 2.2.1
fastjsonschema            2.21.2
fqdn                      1.5.1
h11                       0.16.0
httpcore                  1.0.9
httpx                     0.28.1
idna                      3.11
ipykernel                 7.1.0
ipython                   9.8.0
ipython_pygments_lexers   1.1.1
isoduration               20.11.0
jedi            

## Accessing Environment Variables with `os.getenv()`

- The `os.getenv` function retrieves the value of an environment variable by key, returning None or a provided default if the key is not found.
- It prevents `KeyError` exceptions by offering a safe access pattern for optional configuration settings.
- **Since environment variables are always strings, any expected non-string types require explicit conversion after retrieval.**

In [3]:
import os

#os.environ["APP_KEY"] = "abc123"

api_key = os.getenv("APP_KEY")
debug_mode = os.getenv("DEBUG_MODE", False)

if api_key:
    print(f"API key found: {api_key[:4]}... masked")
else:
    print("APP_KEY not set")

print(f"Debug mode: {debug_mode}")

API key found: abc1... masked
Debug mode: False


## Accessing Environment Variables with `os.environ`

- `os.environ` behaves like a dictionary mapping environment variable names to their string values.
- Accessing a missing key via `os.environ['KEY']` raises a `KeyError`, making it suitable for mandatory variables.
- One should guard against missing keys by checking membership or catching `KeyError` to handle critical configuration errors.

In [8]:
import os

try:
    java_home = os.environ["JAVA_HOME"]
    print(java_home)
except KeyError:
    print("JAVA_HOME not set")

JAVA_HOME not set


## Setting Environment Variables Within Python

- While environment variables are typically set externally, `os.environ` can be modified at runtime to affect the current process and its children.
- Assigning to `os.environ['KEY']` makes the variable available to any subprocesses spawned by the script.
- Deleting an entry from `os.environ` removes it for subsequent operations within the process, but changes do not persist after the script exits.

In [16]:
import os
import sys
import subprocess

print(f"MY_CUSTOM_VAR: {os.getenv("MY_CUSTOM_VAR")}")

os.environ["MY_CUSTOM_VAR"] = "SetMyVAR"
print(f"Updated MY_CUSTOM_VAR: {os.getenv("MY_CUSTOM_VAR")}" )

result = subprocess.run([
    sys.executable,
    "-c",
    """import os
print(f"Child sees MY_CUSTOM_VAR: {os.getenv("MY_CUSTOM_VAR")}")"""
])


del os.environ["MY_CUSTOM_VAR"]
print(f"Updated MY_CUSTOM_VAR: {os.getenv("MY_CUSTOM_VAR")}" )

MY_CUSTOM_VAR: None
Updated MY_CUSTOM_VAR: SetMyVAR
Child sees MY_CUSTOM_VAR: SetMyVAR
Updated MY_CUSTOM_VAR: None


## Using dotenv to Manage Local Environment Files
- The python-dotenv library lets you keep sensitive and environment-specific values in a .env file instead of the shell.
- A .env file lives alongside your script and contains lines like `KEY=value`; it's loaded at runtime into `os.environ`.
- Install with `pip install python-dotenv==1.1.0` (version included here so that we are all using the same, in other installations it may be omitted), then call `load_dotenv()` before any `os.getenv` calls.
- This approach keeps your shell clean and makes it easy to commit example `.env.example` files without secrets.
- Remember not to commit actual .env files with real secrets! Add them to `.gitignore`.

In [26]:
import os
from dotenv import load_dotenv

# To test override
#os.environ["MY_DOTENV_VAR"] = "SetFromJupyter"
#load_dotenv(override=True)

load_dotenv()

secret_value = os.getenv("MY_DOTENV_VAR")
print(f"Get MY_DOTENV_VAR with {secret_value}")

Get MY_DOTENV_VAR with abc123


## Common Pitfalls & How to Avoid Them

- Environment variable names are always case-sensitive in Python, regardless of the underlying OS; inconsistent casing leads to unexpected missing values.
- Forgetting that all environment variable values are strings can cause type errors; always convert to the intended type like int or bool after retrieval.
- Accessing a missing mandatory variable via os.environ raises KeyError; avoid unhandled errors by checking membership or catching exceptions.
- Storing highly sensitive secrets in plain environment variables carries security risks; for production use, consider managed secrets solutions like Vault or AWS Secrets Manager.

In [27]:
import os
from dotenv import load_dotenv


load_dotenv(override=True)

number_dotenv_value = os.getenv("MY_NUMBER_VAR")
print(type(number_dotenv_value))
print(number_dotenv_value)

<class 'str'>
123
